In [5]:
import numpy as np
import scipy as sp 
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
## Constants 
hbar = 1.05*(10**(-34))
pi = 3.14159 
c = 3*(10**(8))
kb = 8.617*(10**(-5))
kb_SI = 1.38*(10**(-23))
qe = 1.6*(10**(-19)) 
eps0 = 8.85*(10**(-12))  
m0 = 9*(10**(-31)) 
a_lat = 3.1*(10**(-10)) 
eV2meter = 1.97*(10**(-7)) ##in meter
eV2sec = 6.58*(10**(-16))
eV2mass = 1.78*(10**(-36))

# Transport physics 
def get_fermi_circle(k_rad, nk, nthet):##ok 
    ##Input: radius of 2D Fermi circle (float), size of k array (int), size of theta array (int)
    ##Output: grid of k points in Fermi circle (array)
    kx_list = []
    ky_list = []
    k_range = np.linspace(0 ,k_rad, int(nk))
    thet_range = np.linspace(0, 2*pi, int(nthet))  
    for i in range(0,nk):
        for j in range(0,nthet): 
            kx_list.append(k_range[i]*np.cos(thet_range[j]))
            ky_list.append(k_range[i]*np.sin(thet_range[j]))
    return [np.asarray(kx_list), np.asarray(ky_list)]  


def get_bose_distribution(energy, T):## ok 
    return (np.exp(energy/(kb_SI*T))-1)**(-1)    

def get_conduction_band(Ec, mstar, k_vec): ##ok 
    ##Input: Conduction band offset (float), effective mass (float), k_point to evaluate at (array)
    ##Output: kmesh (array), conduction band (array)
    Ek = lambda Ec, mstar, kx, ky: Ec + ((hbar**2)/(2*mstar*m0))*(kx**(2) + ky**(2)) 
    return Ek(Ec, mstar, k_vec[0], k_vec[1]) 

def get_group_velocity(mstar, kvec): ##ok 
    ##Input: Conduction band offset (float), effective mass (float),  k_point to evaluate at (array)
    ##Output: kmesh (array), conduction band (array)
    vg = lambda mstar, kvec: [((hbar**(2))/(mstar*m0))*kvec[0], ((hbar**(2))/(mstar*m0))*kvec[1]]
    return np.asarray(vg(mstar, k_vec)) 

def evolvek_h(del_t, E_field, k0):  
    del_k = (qe*del_t*E_field)/hbar 
    return k0 + del_k

def compute_rate_prob_AP(k0, kf, vs, T):  
    q = np.linalg.norm(k0-kf)
    nq = get_bose_distribution(hbar*q*vs, T)
    prob = q*(nq+1)  
    return prob

def compute_rate_prob_POP(k0, kf, mstar, energy_ph):  
    q = np.linalg.norm(k0-kf) 
    E0 = get_conduction_band(0, mstar, k0) 
    Ef = get_conduction_band(0, mstar, kf)
    if((Ef-E0) < energy_ph*qe):
        prob = 0 
    else: 
        prob = q**(-2)
    return prob

def evolvek_h(del_t, E_field, k0): 
    del_k = (qe*del_t/hbar)*E_field 
    return k0 + del_k


def scatterAP_prob_h(k0, kmesh, vs, T):      
    rate_list = np.array([compute_rate_prob_AP(k0, np.array([kmesh[0][i], kmesh[1][i]]) , vs, T) for i in range(len(kmesh[0]))])
    return rate_list/np.sum(rate_list)


def scatterAP_prob_h2(k0, kmesh, mstar, energy_ph):      
    rate_list = np.array([compute_rate_prob_POP(k0, np.array([kmesh[0][i], kmesh[1][i]]) , mstar, energy_ph) for i in range(len(kmesh[0]))])
    rate_list_not_nan = [] 
    kx_list_not_nan = [] 
    ky_list_not_nan = [] 
    for i in range(len(rate_list)):
        if(np.isnan(rate_list[i]) == False):
            rate_list_not_nan.append(rate_list[i])
            kx_list_not_nan.append(kmesh[0][i])
            ky_list_not_nan.append(kmesh[1][i])         
    return [np.array(kx_list_not_nan), np.array(ky_list_not_nan), np.array(rate_list_not_nan)/np.sum(np.array(rate_list_not_nan))]




def scatterPOP_prob_h(k0, kmesh, mstar, energy_ph):      
    rate_list = np.array([compute_rate_prob_POP(k0, np.array([kmesh[0][i], kmesh[1][i]]) , mstar, energy_ph) for i in range(len(kmesh[0]))])
    rate_list_not_nan = [] 
    kx_list_not_nan = [] 
    ky_list_not_nan = [] 
    for i in range(len(rate_list)):
      rate_list_not_nan.append(rate_list[i])
      kx_list_not_nan.append(kmesh[0][i])
      ky_list_not_nan.append(kmesh[1][i])
    return [np.array(kx_list_not_nan), np.array(ky_list_not_nan), np.array(rate_list_not_nan)/np.sum(np.array(rate_list_not_nan))]


def pick_k_h(k0, prob_list):     
    idx = [i for i in range(len(prob_list))]
    idx_ran = np.random.choice(idx, p = prob_list)
    kf = np.asarray([kmesh[0][idx_ran], kmesh[1][idx_ran]]) 
    return kf 


    
def run_MC_AP(kmesh, vs, T, E_field, del_t, Tmax):
    k_list = [] 
    itr = 0      
    k0 = np.array([kmesh[0][0],kmesh[1][0]])
    prob_list = scatterAP_prob_h(k0, kmesh, vs, T)
    while(itr*del_t <= Tmax): 
        ki = evolvek_h(del_t, E_field, k0)       
        k0 = pick_k_h(k0, prob_list) 
        k_list.append(k0.tolist()) 
        itr += 1        
    kx = [k_list[i][0] for i in range(len(k_list))]
    ky = [k_list[i][1] for i in range(len(k_list))]  
    k_vec = [kx, ky]
    return k_vec 

In [6]:
#test MC 
vs = 10**7
a_lat = 3*(10**8)
E_field = np.array([10**(7), 0])
kmesh = get_fermi_circle(2*pi/a_lat, 100, 100)
k_MC = run_MC_AP(kmesh, vs, 300, E_field, 10**(-15), 10**(-12))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in double_scalars
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in double_scalars


ValueError: probabilities contain NaN